In [1]:
# 라이브러리 호출

import pandas as pd
import numpy as np
import re

## 지역1 대시보드 데이터 제작

### 소득구간 종류별 melt 차트용 데이터

In [2]:
# 데이터 호출
data_exp = pd.read_csv("C:/Users/dlwlg/Desktop/final/서울시 상권데이터/서울시_상권분석_서비스(소득소비-상권).csv")

In [3]:
# 데이터에 행정동 및 자치구 데이터 합치기
# 데이터 호출
data_merge = pd.read_csv("C:/Users/dlwlg/Desktop/final/merge.csv", encoding='cp949')
data_merge = data_merge[['상권_코드_명','자치구','행정동']]

# 조인
data_exp_ab = pd.merge(data_exp, data_merge, on='상권_코드_명', how='left')

In [4]:
# 필요한 컬럼 선택
columns_to_keep = ['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명', '자치구', '행정동']
columns_to_melt = ['식료품_지출_총금액', '의류_신발_지출_총금액', '생활용품_지출_총금액', '의료비_지출_총금액', 
                   '교통_지출_총금액', '여가_지출_총금액', '문화_지출_총금액', '교육_지출_총금액', '유흥_지출_총금액']

# 데이터를 긴 형식으로 변환
df_melted_ab = pd.melt(data_exp_ab, id_vars=columns_to_keep, value_vars=columns_to_melt,
                    var_name='지출종류', value_name='지출금액')

# 지출종류 칼럼 정리 (예: '식료품_지출_총금액' -> '식료품')
df_melted_ab['지출종류'] = df_melted_ab['지출종류'].str.extract(r'([가-힣]+)')[0]

In [5]:
# melt 한 데이터 저장 
df_melted_ab.to_csv("C:/Users/dlwlg/Desktop/final/최종 대시보드 데이터/지역/소득소비_melt.csv", encoding='utf-8', index=False)

### 임대시세 데이터에 자치구 / 행정동 라벨링

In [6]:
# 데이터 호출
df_area = pd.read_csv("C:/Users/dlwlg/Desktop/final/merge.csv", encoding='cp949') # 자치구/행정동/상권 데이터
df_price = pd.read_csv("C:/Users/dlwlg/Desktop/final/서울시 상권데이터/서울시상권분석서비스(임대시세-행정동).csv") # 행정동별 임대시세 데이터

In [7]:
# '구분코드'를 문자열로 변환
df_price['구분코드'] = df_price['구분코드'].astype(str)

# '자치구' 컬럼 생성 (구분코드 앞 5자리 기준으로 그룹화)
df_price["자치구"] = df_price.groupby(df_price["구분코드"].str[:5])["행정동_코드_명"].transform("first")

# '행정동' 컬럼 생성 (지역구분이 'dong'일 경우 지역_코드_명을 그대로 복사)
df_price["행정동"] = df_price["행정동_코드_명"].where(df_price["구분"] == "dong", None)

# 결과 확인
df_price.head()


,기준_년분기_코드,구분코드,행정동_코드_명,구분,전체,1층,1층외,자치구,행정동
0,20193,11,서울시 전체,si,117035,133363,100707,서울시 전체,None
1,20193,11110,종로구,gu,192155,225458,158852,종로구,None
2,20193,11110515,청운효자동,dong,122785,154347,91222,종로구,청운효자동
3,20193,11110530,사직동,dong,151287,190743,111830,종로구,사직동
4,20193,11110540,삼청동,dong,194826,161827,227825,종로구,삼청동


In [8]:
# 자치구, 서울시 전체 데이터 drop (행정동 단위 임대시세만 남기기)
df_price = df_price.dropna(subset=['행정동'])

# 널값 확인
df_price.isnull().sum()

기준_년분기_코드    0
구분코드         0
행정동_코드_명     0
구분           0
전체           0
1층           0
1층외          0
자치구          0
행정동          0
dtype: int64

In [9]:
# df_area의 행정동 기준 df_price join
df_ap = pd.merge(df_area, df_price, left_on='행정동', right_on='행정동', how='inner')

# 필요없는 칼럼 제거
columns_to_drop = ['구분코드','행정동_코드_명','구분','자치구_y','상권분류']
df_ap.drop(columns=columns_to_drop, inplace=True)
df_ap.head()

# 칼럼명 변경
df_ap.rename(columns={'자치구_x':'자치구'}, inplace=True)

In [10]:
df_ap

,자치구,행정동,상권_코드_명,기준_년분기_코드,전체,1층,1층외
0,종로구,청운효자동,배화여자대학교(박노수미술관),20193,122785,154347,91222
1,종로구,청운효자동,배화여자대학교(박노수미술관),20202,131133,160242,102023
2,종로구,청운효자동,배화여자대학교(박노수미술관),20211,128011,162329,93692
3,종로구,청운효자동,배화여자대학교(박노수미술관),20214,132428,182188,82669
4,종로구,청운효자동,배화여자대학교(박노수미술관),20223,142725,195183,90268
...,...,...,...,...,...,...,...
37021,강남구,삼성1동,강남 마이스 관광특구,20203,172377,176473,168281
37022,강남구,삼성1동,강남 마이스 관광특구,20212,195150,152134,238166
37023,강남구,삼성1동,강남 마이스 관광특구,20221,202706,179666,225747
37024,강남구,삼성1동,강남 마이스 관광특구,20224,243736,208086,279386


### 데이터 만들기  

#### 1차 데이터 통합

In [11]:
# 데이터 호출
data_all = pd.read_csv("C:/Users/dlwlg/Desktop/final/최종 대시보드 데이터/추정매출.csv")
data_store = pd.read_csv(r"C:\Users\dlwlg\Desktop\final\서울시 상권데이터\서울시_상권분석_서비스(점포-상권).csv")

# 점포 데이터 칼럼명 수정
data_store.rename(columns={'STDR_YYQU_CD':'기준_년분기_코드',
                           'TRDAR_SE_CD':'상권_구분_코드',
                           'TRDAR_SE_CD_NM':'상권_구분_코드_명',
                           'TRDAR_CD':'상권_코드',
                           'TRDAR_CD_NM':'상권_코드_명',
                           'SVC_INDUTY_CD':'서비스_업종_코드',
                           'SVC_INDUTY_CD_NM':'서비스_업종_코드_명',
                           'STOR_CO':'점포_수',
                           'SIMILR_INDUTY_STOR_CO':'유사_업종_점포_수',
                           'OPBIZ_RT':'개업률',
                           'OPBIZ_STOR_CO':'개업_점포_수',
                           'CLSBIZ_RT':'폐업률',
                           'CLSBIZ_STOR_CO':'폐업_점포_수',
                           'FRC_STOR_CO':'프랜차이즈_점포_수'}, inplace=True)
# 사용하지 않는 칼럼 drop (코드)
columns_to_drop_all = ['상권_구분_코드','상권_코드','서비스_업종_코드']
data_all.drop(columns=columns_to_drop_all, inplace = True)
data_store.drop(columns=columns_to_drop_all, inplace = True)

In [12]:
# 만든 데이터에 임대시세 추가
merged_s_ap = pd.merge(data_all, df_ap, on = ['기준_년분기_코드','상권_코드_명'],how='left')

In [13]:
# 만든 데이터에 점포 수 추가
merged_final = pd.merge(merged_s_ap, data_store, on = ['기준_년분기_코드','상권_코드_명','서비스_업종_코드_명'],how='left')

In [14]:
# 매출액 데이터 따로 지정
drop_cols1 = merged_final.loc[:, "주중_매출_건수":"연령대_60_이상_매출_건수"].columns 
sale = merged_final.drop(columns=drop_cols1)

# 칼럼명 간단하게 수정(주중_매출_금액 > 주중)
rename_dict = {
    **{f"{d}_매출_금액": d for d in ["주중", "주말", "월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일"]},
    **{f"시간대_{t}_매출_금액": t for t in ["00~06", "06~11", "11~14", "14~17", "17~21", "21~24"]},
    **{f"{g}_매출_금액": g for g in ["남성", "여성"]},
    **{f"연령대_{a}_매출_금액": f"{a}대" for a in ["10", "20", "30", "40", "50"]},
    "연령대_60_이상_매출_금액": "60대 이상",
    "유사_업종_점포_수" : "전체 점포 수",
    "개업_점포_수" : "개업 점포 수",
    "폐업_점포_수" : "폐업 점포 수",
    "프랜차이즈_점포_수" : "프랜차이즈 점포 수"
}

sale.rename(columns=rename_dict, inplace=True)

In [15]:
# 건수 및 점포수 데이터 따로 지정
drop_cols2 = merged_final.loc[:, "주중_매출_금액":"연령대_60_이상_매출_금액"].columns 
count = merged_final.drop(columns=drop_cols2)

# 칼럼명 간단하게 수정(주중_매출_건수 > 주중)
rename_dict = {
    **{f"{d}_매출_건수": d for d in ["주중", "주말", "월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일"]},
    **{f"시간대_{t}_매출_건수": t for t in ["00~06", "06~11", "11~14", "14~17", "17~21", "21~24"]},
    **{f"{g}_매출_건수": g for g in ["남성", "여성"]},
    **{f"연령대_{a}_매출_건수": f"{a}대" for a in ["10", "20", "30", "40", "50"]},
    "연령대_60_이상_매출_건수": "60대 이상",
    "유사_업종_점포_수" : "전체 점포 수",
    "개업_점포_수" : "개업 점포 수",
    "폐업_점포_수" : "폐업 점포 수",
    "프랜차이즈_점포_수" : "프랜차이즈 점포 수"
}

count.rename(columns=rename_dict, inplace=True)

#### 성별 / 연령대 / 요일 / 시간대 melt

In [16]:
# 고정할 컬럼 (melt 시 기준이 되는 컬럼)
id_vars1 = ['기준_년분기_코드', '상권_코드_명', '서비스_업종_코드_명', '전체', '1층', '1층외', '행정동', '자치구', '당월_매출_금액']
id_vars2 = ['기준_년분기_코드', '상권_코드_명', '서비스_업종_코드_명', '전체', '1층', '1층외', '행정동', '자치구', '당월_매출_건수','당월_매출_금액',
           "전체 점포 수","개업 점포 수",'폐업 점포 수','프랜차이즈 점포 수']
# melt되는 컬럼
sales_vars = ["주중", "주말", "월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일",
              "00~06", "06~11", "11~14", "14~17", "17~21", "21~24",
              "남성", "여성",
              "10대", "20대", "30대", "40대", "50대","60대 이상"]

# 매출, 건수 데이터 melt
df_sales_melted = sale.melt(id_vars=id_vars1, value_vars=sales_vars, 
                           var_name='구분', value_name='매출_값')
df_count_melted = count.melt(id_vars=id_vars2, value_vars=sales_vars, 
                           var_name='구분', value_name='건수_값')


In [17]:
# 중복 제거 
df_sales_melted.drop_duplicates(subset=['기준_년분기_코드', '상권_코드_명', '서비스_업종_코드_명', '구분'], inplace=True)
df_count_melted.drop_duplicates(subset=['기준_년분기_코드', '상권_코드_명', '서비스_업종_코드_명', '구분'], inplace=True)

In [18]:
# 매출액, 매출건수 데이터 고정된 칼럼과 '구분' 칼럼 기준으로 join 
df_merged = pd.merge(df_sales_melted, df_count_melted, on=id_vars1 + ['구분'], how='left')

# 시간대 데이터 알기쉽게 수정 (00~06 > 00시~06시) 
# 변환 함수 정의
def convert_time_format(value):
    if re.match(r'^\d{2}~\d{2}$', value):  # 숫자 두 개~숫자 두 개 패턴만 변환
        return value.replace('~', '시~') + '시'
    return value  # 그 외의 값은 그대로 반환

# '구분' 칼럼 변환
df_merged['구분'] = df_merged['구분'].apply(convert_time_format)

# 결과 확인
df_merged.head()

,기준_년분기_코드,상권_코드_명,서비스_업종_코드_명,전체,1층,1층외,행정동,자치구,당월_매출_금액,구분,매출_값,당월_매출_건수,전체 점포 수,개업 점포 수,폐업 점포 수,프랜차이즈 점포 수,건수_값
0,20222,영등포유통상가,완구,90731,127868,53593,당산1동,영등포구,2.669903e+07,주중,0.000000e+00,243,14,0,0,0,0
1,20222,당고개역 2번,철물점,118445,100126,136764,상계3·4동,노원구,2.739245e+06,주중,2.739245e+06,46,3,0,0,0,46
2,20222,신길2동우편취급국,의료기기,104170,112800,95540,영등포본동,영등포구,2.192000e+09,주중,2.192000e+09,284,8,0,0,0,284
3,20222,수송초등학교,가구,100854,104548,97161,번1동,강북구,4.087973e+07,주중,4.087973e+07,21,3,0,0,0,21
4,20222,창동신창시장,의료기기,95211,80410,110012,창2동,도봉구,4.893243e+07,주중,0.000000e+00,98,3,0,0,0,0


#### 지역1 대시보드 상단 데이터 만들기 

In [19]:
# 상단 데이터에 쓸 칼럼 
chart1_columns = ['기준_년분기_코드','상권_코드_명','행정동','자치구','전체','1층','1층외','당월_매출_금액']
chart1 = df_merged[chart1_columns].copy()

In [20]:
# 20242 데이터만 남기기
chart1 = chart1[chart1['기준_년분기_코드'].isin([20242])]

sale.rename(columns=rename_dict, inplace=True)

In [21]:
# 상권활성화지표 데이터 불러오기
data_active = pd.read_csv("C:/Users/dlwlg/Desktop/final/서울시 상권데이터/서울시_상권분석_서비스(상권활성화지표-상권).csv")

In [22]:
# 기준년분기, 상권코드명 기준으로 상권 변화 지표명, 운영/폐업 개월 평균 붙이기 
final_chart1 = pd.merge(chart1, data_active[['기준_년분기_코드','상권_코드_명','상권_변화_지표_명','운영_영업_개월_평균','폐업_영업_개월_평균']], 
                         on=['상권_코드_명','기준_년분기_코드'], how = 'left')

In [23]:
# 데이터 저장
final_chart1.to_csv("C:/Users/dlwlg/Desktop/final/최종 대시보드 데이터/지역/지역1 대시보드 상단 데이터.csv", encoding='utf-8', index=False)

#### 지역1 대시보드 하단 데이터 만들기

In [24]:
# 제외할 컬럼 선택 
chart2_columns = ['전체','1층','1층외']
chart2 = df_merged.drop(columns=chart2_columns)

In [25]:
# 20242 데이터만 남겨둠
filtered_chart2 = chart2[chart2['기준_년분기_코드'].isin([20241, 20242])]

# '기준_년분기_코드' 값을 년월로 변환하는 함수 정의
def convert_to_date(code):
    year = int(str(code)[:4])  # 연도 추출
    quarter = int(str(code)[-1])  # 분기 추출
    month = (quarter - 1) * 3 + 1  # 분기에 따라 시작 월 계산
    return pd.Timestamp(year=year, month=month, day=1)
# 변환 적용
filtered_chart2['기준_년분기_코드_날짜'] = filtered_chart2['기준_년분기_코드'].apply(convert_to_date)

C:\Users\dlwlg\AppData\Local\Temp\ipykernel_19724\3943729691.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_chart2['기준_년분기_코드_날짜'] = filtered_chart2['기준_년분기_코드'].apply(convert_to_date)


In [26]:
filtered_chart2 = filtered_chart2.drop(columns=['개업 점포 수','폐업 점포 수']).copy()

In [27]:
# 중복 제거 
filtered_chart2.drop_duplicates(subset=['기준_년분기_코드', '상권_코드_명', '서비스_업종_코드_명', '구분'], inplace=True)

In [28]:
final_chart2 = (
    filtered_chart2.groupby(['기준_년분기_코드', '상권_코드_명', '구분'])
    [['매출_값', '건수_값', '전체 점포 수', '프랜차이즈 점포 수', '당월_매출_금액', '당월_매출_건수']]
    .sum()
    .reset_index()
)

In [29]:
final_chart2

,기준_년분기_코드,상권_코드_명,구분,매출_값,건수_값,전체 점포 수,프랜차이즈 점포 수,당월_매출_금액,당월_매출_건수
0,20241,4.19민주묘지역 2번,00시~06시,98104897.0,7771,43,6,2.457760e+09,127351
1,20241,4.19민주묘지역 2번,06시~11시,124462531.0,14332,43,6,2.457760e+09,127351
2,20241,4.19민주묘지역 2번,10대,7653212.0,950,43,6,2.457760e+09,127351
3,20241,4.19민주묘지역 2번,11시~14시,532747163.0,26832,43,6,2.457760e+09,127351
4,20241,4.19민주묘지역 2번,14시~17시,425544848.0,23916,43,6,2.457760e+09,127351
...,...,...,...,...,...,...,...,...,...
68489,20242,흑석시장,일요일,9694947.0,439,14,0,2.879912e+08,14208
68490,20242,흑석시장,주말,48726154.0,2506,14,0,2.879912e+08,14208
68491,20242,흑석시장,주중,239265034.0,11702,14,0,2.879912e+08,14208
68492,20242,흑석시장,토요일,39031207.0,2067,14,0,2.879912e+08,14208


In [30]:
final_chart2.to_csv("C:/Users/dlwlg/Desktop/final/최종 대시보드 데이터/지역/지역1 대시보드 하단 데이터.csv", encoding='utf-8', index=False)

## 지역2 대시보드 제작

### 유동인구 melt

In [31]:
# 데이터 호출
data_people = pd.read_csv("C:/Users/dlwlg/Desktop/final/서울시 상권데이터/서울시_상권분석_서비스(길단위인구-상권).csv")

In [32]:
# 칼럼명 간단하게 수정(월요일_유동인구_수 > 월요일) 
rename_dict = {
    **{f"{d}_유동인구_수": d for d in ["월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일"]},
    **{f"시간대_{t}_유동인구_수": t for t in ["00_06", "06_11", "11_14", "14_17", "17_21", "21_24"]},
    **{f"{g}_유동인구_수": g for g in ["남성", "여성"]},
    **{f"연령대_{a}_유동인구_수": f"{a}대" for a in ["10", "20", "30", "40", "50"]},
    "연령대_60_이상_유동인구_수": "60대 이상"
}

data_people.rename(columns=rename_dict, inplace=True)

In [33]:
# 고정할 컬럼 (melt 시 기준이 되는 컬럼)
id_vars = ['기준_년분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '총_유동인구_수']

# 유동인구 관련 컬럼 분리
flow_vars = [
    "남성", "여성",
    "10대",'20대','30대','40대','50대','60대 이상',
    "00_06",'06_11','11_14','14_17','17_21','21_24',
    "월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일"
]

# 데이터 melt
df_melted = data_people.melt(id_vars=id_vars, value_vars=flow_vars, var_name='구분', value_name='유동인구_수')

# 시간대 데이터 알기쉽게 수정 (00_06 > 00시~06시) 
# 변환 함수 정의
def convert_time_format(value):
    if re.match(r'^\d{2}_\d{2}$', value):  # 숫자 두 개~숫자 두 개 패턴만 변환
        return value.replace('_', '시~') + '시'
    return value  # 그 외의 값은 그대로 반환

# '구분' 칼럼 변환
df_melted['구분'] = df_melted['구분'].apply(convert_time_format)

# 결과 확인
df_melted.head()

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_유동인구_수,구분,유동인구_수
0,20211,R,전통시장,3130057,만리시장,48523,남성,23552
1,20241,A,골목상권,3110412,창동중학교,151994,남성,71222
2,20241,A,골목상권,3110476,역촌역 2번,78314,남성,36795
3,20211,R,전통시장,3130093,동서시장,55869,남성,28962
4,20212,R,전통시장,3130124,번동시장,18760,남성,8687


In [34]:
# 20241, 20242 데이터만 남기기 
final_chart3 = df_melted[df_melted['기준_년분기_코드'].isin([20241, 20242])]

# 필요없는 칼럼 제외
drop_cols = ['상권_구분_코드','상권_구분_코드_명','상권_코드']
final_chart3.drop(columns=drop_cols, inplace=True)

# 데이터 확인
final_chart3.head()

C:\Users\dlwlg\AppData\Local\Temp\ipykernel_19724\1306962243.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_chart3.drop(columns=drop_cols, inplace=True)


,기준_년분기_코드,상권_코드_명,총_유동인구_수,구분,유동인구_수
1,20241,창동중학교,151994,남성,71222
2,20241,역촌역 2번,78314,남성,36795
14,20241,역삼역 4번,635129,남성,335416
19,20241,63빌딩,62134,남성,31901
25,20241,방산종합시장(방산시장),950124,남성,568483


In [35]:
final_chart3.to_csv("C:/Users/dlwlg/Desktop/final/최종 대시보드 데이터/지역/유동인구.csv", encoding='utf-8', index=False)

### 집객시설 데이터 만들기

In [36]:
# 데이터 호출
data_place = pd.read_csv("C:/Users/dlwlg/Desktop/final/최종 대시보드 데이터/지역/집객시설_수정.csv")

In [37]:
# 행정동, 자치구 데이터 붙이기

merged_place = pd.merge(data_place,data_merge, on='상권_코드_명', how='left')

In [38]:
merged_place.to_csv("C:/Users/dlwlg/Desktop/final/최종 대시보드 데이터/지역/집객시설 최종.csv", encoding='utf-8', index=False)

## 상권 통합 대시보드 데이터 제작

### MELT 데이터 제작

In [39]:
final_chart4 = pd.merge(final_chart2, final_chart3, on=['기준_년분기_코드','상권_코드_명','구분'], how='left')
final_chart4 = pd.merge(final_chart4, data_merge, on='상권_코드_명', how='left')
# '기준_년분기_코드' 값을 년월로 변환하는 함수 정의
def convert_to_date(code):
    year = int(str(code)[:4])  # 연도 추출
    quarter = int(str(code)[-1])  # 분기 추출
    month = (quarter - 1) * 3 + 1  # 분기에 따라 시작 월 계산
    return pd.Timestamp(year=year, month=month, day=1)
# 변환 적용
final_chart4['기준_년분기_코드_날짜'] = final_chart4['기준_년분기_코드'].apply(convert_to_date)

# 데이터 확인
final_chart4.head()

,기준_년분기_코드,상권_코드_명,구분,매출_값,건수_값,전체 점포 수,프랜차이즈 점포 수,당월_매출_금액,당월_매출_건수,총_유동인구_수,유동인구_수,자치구,행정동,기준_년분기_코드_날짜
0,20241,4.19민주묘지역 2번,00시~06시,98104897.0,7771,43,6,2.457760e+09,127351,1021175.0,307497.0,강북구,우이동,2024-01-01
1,20241,4.19민주묘지역 2번,06시~11시,124462531.0,14332,43,6,2.457760e+09,127351,1021175.0,196319.0,강북구,우이동,2024-01-01
2,20241,4.19민주묘지역 2번,10대,7653212.0,950,43,6,2.457760e+09,127351,1021175.0,136192.0,강북구,우이동,2024-01-01
3,20241,4.19민주묘지역 2번,11시~14시,532747163.0,26832,43,6,2.457760e+09,127351,1021175.0,114663.0,강북구,우이동,2024-01-01
4,20241,4.19민주묘지역 2번,14시~17시,425544848.0,23916,43,6,2.457760e+09,127351,1021175.0,108147.0,강북구,우이동,2024-01-01


In [40]:
final_chart4 = final_chart4[~final_chart4['구분'].isin(['주말', '주중'])]
final_chart4

,기준_년분기_코드,상권_코드_명,구분,매출_값,건수_값,전체 점포 수,프랜차이즈 점포 수,당월_매출_금액,당월_매출_건수,총_유동인구_수,유동인구_수,자치구,행정동,기준_년분기_코드_날짜
0,20241,4.19민주묘지역 2번,00시~06시,98104897.0,7771,43,6,2.457760e+09,127351,1021175.0,307497.0,강북구,우이동,2024-01-01
1,20241,4.19민주묘지역 2번,06시~11시,124462531.0,14332,43,6,2.457760e+09,127351,1021175.0,196319.0,강북구,우이동,2024-01-01
2,20241,4.19민주묘지역 2번,10대,7653212.0,950,43,6,2.457760e+09,127351,1021175.0,136192.0,강북구,우이동,2024-01-01
3,20241,4.19민주묘지역 2번,11시~14시,532747163.0,26832,43,6,2.457760e+09,127351,1021175.0,114663.0,강북구,우이동,2024-01-01
4,20241,4.19민주묘지역 2번,14시~17시,425544848.0,23916,43,6,2.457760e+09,127351,1021175.0,108147.0,강북구,우이동,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68947,20242,흑석시장,여성,106296650.0,6427,14,0,2.879912e+08,14208,85841.0,46613.0,동작구,흑석동,2024-04-01
68948,20242,흑석시장,월요일,61379295.0,2306,14,0,2.879912e+08,14208,85841.0,12420.0,동작구,흑석동,2024-04-01
68949,20242,흑석시장,일요일,9694947.0,439,14,0,2.879912e+08,14208,85841.0,10685.0,동작구,흑석동,2024-04-01
68952,20242,흑석시장,토요일,39031207.0,2067,14,0,2.879912e+08,14208,85841.0,12040.0,동작구,흑석동,2024-04-01


In [41]:
# 데이터 저장
final_chart4.to_csv("C:/Users/dlwlg/Desktop/final/최종 대시보드 데이터/지역/최종상권하단데이터.csv", encoding='utf-8',index=False)

In [50]:
# 평균과의 비교 위한 데이터
# 행정동별 집계
df_dong = final_chart4.groupby(['기준_년분기_코드', '행정동'])[["당월_매출_금액", "당월_매출_건수", "총_유동인구_수"]].mean().reset_index()
df_dong = df_dong.rename(columns={
    "당월_매출_금액": "행정동_당월매출금액", "당월_매출_건수": "행정동_당월매출건수", "총_유동인구_수": "행정동_총유동인구수"
})

# 자치구별 집계
df_gu = final_chart4.groupby(['기준_년분기_코드', '자치구'])[["당월_매출_금액", "당월_매출_건수", "총_유동인구_수"]].mean().reset_index()
df_gu = df_gu.rename(columns={
    "당월_매출_금액": "자치구_당월매출금액", "당월_매출_건수": "자치구_당월매출건수", "총_유동인구_수": "자치구_총유동인구수"
})

# 서울시 전체 집계
df_seoul = final_chart4.groupby('기준_년분기_코드')[["당월_매출_금액", "당월_매출_건수", "총_유동인구_수"]].mean().reset_index()
df_seoul = df_seoul.rename(columns={
    "당월_매출_금액": "서울시_당월매출금액", "당월_매출_건수": "서울시_당월매출건수", "총_유동인구_수": "서울시_총유동인구수"
})

# 원본 데이터와 병합
final_df = final_chart4.merge(df_dong, on="행정동", how="left")
final_df = final_df.merge(df_gu, on="자치구", how="left")
final_df = pd.merge(final_df, df_seoul, on="기준_년분기_코드", how="left")
# 결과 확인
final_df


,기준_년분기_코드_x,상권_코드_명,구분,매출_값,건수_값,전체 점포 수,프랜차이즈 점포 수,당월_매출_금액,당월_매출_건수,총_유동인구_수,...,행정동_당월매출금액,행정동_당월매출건수,행정동_총유동인구수,기준_년분기_코드,자치구_당월매출금액,자치구_당월매출건수,자치구_총유동인구수,서울시_당월매출금액,서울시_당월매출건수,서울시_총유동인구수
0,20241,4.19민주묘지역 2번,00시~06시,98104897.0,7771,43,6,2.457760e+09,127351,1021175.0,...,2.705026e+09,140052.333333,483254.000000,20241,5.626987e+09,237058.140351,933670.578947,1.516698e+10,504882.708472,900538.171562
1,20241,4.19민주묘지역 2번,00시~06시,98104897.0,7771,43,6,2.457760e+09,127351,1021175.0,...,2.705026e+09,140052.333333,483254.000000,20242,6.032127e+09,263495.824561,915943.859649,1.538637e+10,551962.911274,887936.767023
2,20241,4.19민주묘지역 2번,00시~06시,98104897.0,7771,43,6,2.457760e+09,127351,1021175.0,...,3.167201e+09,164331.666667,460936.666667,20241,5.626987e+09,237058.140351,933670.578947,1.516698e+10,504882.708472,900538.171562
3,20241,4.19민주묘지역 2번,00시~06시,98104897.0,7771,43,6,2.457760e+09,127351,1021175.0,...,3.167201e+09,164331.666667,460936.666667,20242,6.032127e+09,263495.824561,915943.859649,1.538637e+10,551962.911274,887936.767023
4,20241,4.19민주묘지역 2번,06시~11시,124462531.0,14332,43,6,2.457760e+09,127351,1021175.0,...,2.705026e+09,140052.333333,483254.000000,20241,5.626987e+09,237058.140351,933670.578947,1.516698e+10,504882.708472,900538.171562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251827,20242,흑석시장,토요일,39031207.0,2067,14,0,2.879912e+08,14208,85841.0,...,4.895205e+09,330884.666667,490118.333333,20242,2.304795e+10,606221.340000,818054.840000,1.538637e+10,551962.911274,887936.767023
251828,20242,흑석시장,화요일,36210975.0,2207,14,0,2.879912e+08,14208,85841.0,...,4.071062e+09,255396.333333,441426.333333,20241,2.650528e+10,599874.520000,821947.100000,1.516698e+10,504882.708472,900538.171562
251829,20242,흑석시장,화요일,36210975.0,2207,14,0,2.879912e+08,14208,85841.0,...,4.071062e+09,255396.333333,441426.333333,20242,2.304795e+10,606221.340000,818054.840000,1.538637e+10,551962.911274,887936.767023
251830,20242,흑석시장,화요일,36210975.0,2207,14,0,2.879912e+08,14208,85841.0,...,4.895205e+09,330884.666667,490118.333333,20241,2.650528e+10,599874.520000,821947.100000,1.516698e+10,504882.708472,900538.171562


In [ ]:
final_df = final_df[final_df["기준_년분기_코드_x"] == final_df["기준_년분기_코드"]]
# 필요 없는 `_x`, `_y` 컬럼 정리
final_df = final_df.drop(columns=["기준_년분기_코드"])
final_df = final_df.rename(columns={"기준_년분기_코드_x": "기준_년분기_코드"})

# 데이터 확인
final_df

,기준_년분기_코드,상권_코드_명,구분,매출_값,건수_값,전체 점포 수,프랜차이즈 점포 수,당월_매출_금액,당월_매출_건수,총_유동인구_수,...,기준_년분기_코드_y,행정동_당월매출금액,행정동_당월매출건수,행정동_총유동인구수,자치구_당월매출금액,자치구_당월매출건수,자치구_총유동인구수,서울시_당월매출금액,서울시_당월매출건수,서울시_총유동인구수
0,20241,4.19민주묘지역 2번,00시~06시,98104897.0,7771,43,6,2.457760e+09,127351,1021175.0,...,20241,2.705026e+09,140052.333333,483254.000000,5.626987e+09,237058.140351,933670.578947,1.516698e+10,504882.708472,900538.171562
2,20241,4.19민주묘지역 2번,00시~06시,98104897.0,7771,43,6,2.457760e+09,127351,1021175.0,...,20242,3.167201e+09,164331.666667,460936.666667,5.626987e+09,237058.140351,933670.578947,1.516698e+10,504882.708472,900538.171562
4,20241,4.19민주묘지역 2번,06시~11시,124462531.0,14332,43,6,2.457760e+09,127351,1021175.0,...,20241,2.705026e+09,140052.333333,483254.000000,5.626987e+09,237058.140351,933670.578947,1.516698e+10,504882.708472,900538.171562
6,20241,4.19민주묘지역 2번,06시~11시,124462531.0,14332,43,6,2.457760e+09,127351,1021175.0,...,20242,3.167201e+09,164331.666667,460936.666667,5.626987e+09,237058.140351,933670.578947,1.516698e+10,504882.708472,900538.171562
8,20241,4.19민주묘지역 2번,10대,7653212.0,950,43,6,2.457760e+09,127351,1021175.0,...,20241,2.705026e+09,140052.333333,483254.000000,5.626987e+09,237058.140351,933670.578947,1.516698e+10,504882.708472,900538.171562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251823,20242,흑석시장,일요일,9694947.0,439,14,0,2.879912e+08,14208,85841.0,...,20242,4.895205e+09,330884.666667,490118.333333,2.304795e+10,606221.340000,818054.840000,1.538637e+10,551962.911274,887936.767023
251825,20242,흑석시장,토요일,39031207.0,2067,14,0,2.879912e+08,14208,85841.0,...,20241,4.071062e+09,255396.333333,441426.333333,2.304795e+10,606221.340000,818054.840000,1.538637e+10,551962.911274,887936.767023
251827,20242,흑석시장,토요일,39031207.0,2067,14,0,2.879912e+08,14208,85841.0,...,20242,4.895205e+09,330884.666667,490118.333333,2.304795e+10,606221.340000,818054.840000,1.538637e+10,551962.911274,887936.767023
251829,20242,흑석시장,화요일,36210975.0,2207,14,0,2.879912e+08,14208,85841.0,...,20241,4.071062e+09,255396.333333,441426.333333,2.304795e+10,606221.340000,818054.840000,1.538637e+10,551962.911274,887936.767023


In [53]:
final_df = final_df[final_df["기준_년분기_코드_y"] == final_df["기준_년분기_코드"]]
# 필요 없는 `_x`, `_y` 컬럼 정리
final_df = final_df.drop(columns=["기준_년분기_코드_y"])

# 데이터 확인
final_df

,기준_년분기_코드,상권_코드_명,구분,매출_값,건수_값,전체 점포 수,프랜차이즈 점포 수,당월_매출_금액,당월_매출_건수,총_유동인구_수,...,기준_년분기_코드_날짜,행정동_당월매출금액,행정동_당월매출건수,행정동_총유동인구수,자치구_당월매출금액,자치구_당월매출건수,자치구_총유동인구수,서울시_당월매출금액,서울시_당월매출건수,서울시_총유동인구수
0,20241,4.19민주묘지역 2번,00시~06시,98104897.0,7771,43,6,2.457760e+09,127351,1021175.0,...,2024-01-01,2.705026e+09,140052.333333,483254.000000,5.626987e+09,237058.140351,933670.578947,1.516698e+10,504882.708472,900538.171562
4,20241,4.19민주묘지역 2번,06시~11시,124462531.0,14332,43,6,2.457760e+09,127351,1021175.0,...,2024-01-01,2.705026e+09,140052.333333,483254.000000,5.626987e+09,237058.140351,933670.578947,1.516698e+10,504882.708472,900538.171562
8,20241,4.19민주묘지역 2번,10대,7653212.0,950,43,6,2.457760e+09,127351,1021175.0,...,2024-01-01,2.705026e+09,140052.333333,483254.000000,5.626987e+09,237058.140351,933670.578947,1.516698e+10,504882.708472,900538.171562
12,20241,4.19민주묘지역 2번,11시~14시,532747163.0,26832,43,6,2.457760e+09,127351,1021175.0,...,2024-01-01,2.705026e+09,140052.333333,483254.000000,5.626987e+09,237058.140351,933670.578947,1.516698e+10,504882.708472,900538.171562
16,20241,4.19민주묘지역 2번,14시~17시,425544848.0,23916,43,6,2.457760e+09,127351,1021175.0,...,2024-01-01,2.705026e+09,140052.333333,483254.000000,5.626987e+09,237058.140351,933670.578947,1.516698e+10,504882.708472,900538.171562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251815,20242,흑석시장,여성,106296650.0,6427,14,0,2.879912e+08,14208,85841.0,...,2024-04-01,4.895205e+09,330884.666667,490118.333333,2.304795e+10,606221.340000,818054.840000,1.538637e+10,551962.911274,887936.767023
251819,20242,흑석시장,월요일,61379295.0,2306,14,0,2.879912e+08,14208,85841.0,...,2024-04-01,4.895205e+09,330884.666667,490118.333333,2.304795e+10,606221.340000,818054.840000,1.538637e+10,551962.911274,887936.767023
251823,20242,흑석시장,일요일,9694947.0,439,14,0,2.879912e+08,14208,85841.0,...,2024-04-01,4.895205e+09,330884.666667,490118.333333,2.304795e+10,606221.340000,818054.840000,1.538637e+10,551962.911274,887936.767023
251827,20242,흑석시장,토요일,39031207.0,2067,14,0,2.879912e+08,14208,85841.0,...,2024-04-01,4.895205e+09,330884.666667,490118.333333,2.304795e+10,606221.340000,818054.840000,1.538637e+10,551962.911274,887936.767023


In [55]:
# 데이터 저장
final_df.to_csv("C:/Users/dlwlg/Desktop/final/최종 대시보드 데이터/지역/최종상권평균비교데이터.csv", encoding='utf-8',index=False)